In [1]:
!nvidia-smi

Tue Dec 30 08:14:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
%%writefile quick_sort.cu

#include <iostream>           // Для cout
#include <vector>             // Для std::vector
#include <cstdlib>            // Для rand()
#include <chrono>             // Для измерения времени
#include <cuda_runtime.h>     // Для работы с CUDA API

using namespace std;          // Чтобы не писать std::

// GPU: сортировка вставкой
// Используется для небольших подмассивов внутри потоков
__device__ void insertionSort(int* arr, int left, int right) { // Сортировка вставкой на GPU
    for (int i = left + 1; i <= right; i++) {                 // Проходим по элементам подмассива
        int key = arr[i];                                      // Текущий элемент
        int j = i - 1;
        while (j >= left && arr[j] > key) {                    // Сдвигаем элементы вправо
            arr[j + 1] = arr[j];
            j--;
        }
        arr[j + 1] = key;                                     // Вставляем элемент на своё место
    }
}

//  Kernel
// Каждый поток сортирует свой подмассив
__global__ void quickSortKernel(int* arr, int size, int subSize) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс потока
    int start = tid * subSize;                       // Начало подмассива
    int end = min(start + subSize - 1, size - 1);   // Конец подмассива
    if (start < size)                                // Проверка границ
        insertionSort(arr, start, end);             // Сортируем подмассив
}

// Main
int main() {
    vector<int> sizes = {10000, 100000};             // Размеры массивов

    for (int s = 0; s < sizes.size(); ++s) {
        int size = sizes[s];
        vector<int> h_arr(size);                     // Массив на CPU

        for (int i = 0; i < size; i++)               // Заполняем случайными числами
            h_arr[i] = rand() % 100000;

        cout << "Исходный массив (первые 20 элементов): ";
        for (int i = 0; i < 20 && i < size; i++) cout << h_arr[i] << " "; // Вывод первых 20 элементов
        cout << endl;

        int* d_arr;
        cudaMalloc(&d_arr, size * sizeof(int));     // Выделяем память на GPU
        cudaMemcpy(d_arr, h_arr.data(), size * sizeof(int), cudaMemcpyHostToDevice); // Копируем данные

        int threadsPerBlock = 256;                  // Потоки в блоке
        int subArraySize = 1024;                    // Размер подмассива для одного потока
        int numBlocks = (size + subArraySize - 1) / subArraySize; // Кол-во блоков

        auto startTime = chrono::high_resolution_clock::now(); // Начало измерения времени

        quickSortKernel<<<numBlocks, threadsPerBlock>>>(d_arr, size, subArraySize); // Запуск kernel
        cudaDeviceSynchronize();                                  // Ждем завершения

        auto endTime = chrono::high_resolution_clock::now();     // Конец измерения времени
        cudaMemcpy(h_arr.data(), d_arr, size * sizeof(int), cudaMemcpyDeviceToHost); // Копируем результат

        chrono::duration<double> duration = endTime - startTime; // Вычисляем время

        cout << "Отсортированный массив (первые 20 элементов): ";
        for (int i = 0; i < 20 && i < size; i++) cout << h_arr[i] << " "; // Вывод первых 20 элементов
        cout << endl;

        cout << "Размер массива: " << size << endl;
        cout << "Время GPU сортировки: " << duration.count() << " секунд" << endl;

        cudaFree(d_arr);                                       // Освобождаем память GPU

    }

    return 0;                                                   // Завершение программы
}


Writing quick_sort.cu


In [2]:
# Компиляция и запуск в Colab
!nvcc quick_sort.cu -o quick_sort
!./quick_sort

Исходный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Отсортированный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Размер массива: 10000
Время GPU сортировки: 0.048958 секунд
Исходный массив (первые 20 элементов): 57537 48410 73756 77667 85312 32062 54136 67229 56846 9902 28956 14752 73853 84999 96547 22245 14905 59807 86594 83387 
Отсортированный массив (первые 20 элементов): 57537 48410 73756 77667 85312 32062 54136 67229 56846 9902 28956 14752 73853 84999 96547 22245 14905 59807 86594 83387 
Размер массива: 100000
Время GPU сортировки: 3.9429e-05 секунд
